In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import string
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from transformers import BertTokenizer, TFBertModel


In [ ]:
# Load the dataset
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines=True)

In [ ]:
# Preprocess the text
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text


In [ ]:
# Apply the preprocessing function to the 'headline' column
data['headline'] = data['headline'].apply(preprocess_text)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['headline'], data['is_sarcastic'], test_size=0.3, random_state=42)

In [ ]:
# Tokenize the text using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

X_train_seq = tokenizer.batch_encode_plus(
    X_train.tolist(),
    truncation=True,
    max_length=100,
    padding='max_length',
    return_token_type_ids=False
)

X_test_seq = tokenizer.batch_encode_plus(
    X_test.tolist(),
    truncation=True,
    max_length=100,
    padding='max_length',
    return_token_type_ids=False
)

In [ ]:
# Convert tokenized sequences to numpy arrays
X_train_pad = np.array(X_train_seq['input_ids'])
X_test_pad = np.array(X_test_seq['input_ids'])


In [ ]:
# Build the model
model = Sequential()
model.add(Embedding(len(tokenizer), 100, input_length=100, trainable=False))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Define early stopping criteria
#early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train the model with early stopping
history = model.fit(X_train_pad, y_train, epochs=35, batch_size=32, validation_split=0.2)

Epoch 1/35
468/468 [==============================] - 23s 28ms/step - loss: 0.6208 - accuracy: 0.6423 - val_loss: 0.5722 - val_accuracy: 0.6896
Epoch 2/35
468/468 [==============================] - 13s 27ms/step - loss: 0.5782 - accuracy: 0.6859 - val_loss: 0.5566 - val_accuracy: 0.7061
Epoch 3/35
468/468 [==============================] - 11s 23ms/step - loss: 0.5520 - accuracy: 0.7022 - val_loss: 0.5409 - val_accuracy: 0.7142
Epoch 4/35
468/468 [==============================] - 10s 22ms/step - loss: 0.5330 - accuracy: 0.7166 - val_loss: 0.5356 - val_accuracy: 0.7126
Epoch 5/35
468/468 [==============================] - 11s 22ms/step - loss: 0.5208 - accuracy: 0.7270 - val_loss: 0.5289 - val_accuracy: 0.7249
Epoch 6/35
468/468 [==============================] - 11s 24ms/step - loss: 0.5040 - accuracy: 0.7380 - val_loss: 0.5273 - val_accuracy: 0.7374
Epoch 7/35
468/468 [==============================] - 11s 24ms/step - loss: 0.4938 - accuracy: 0.7479 - val_loss: 0.5231 - val_accuracy:

In [ ]:
# Evaluate the model
y_pred_prob = model.predict(X_test_pad)
y_pred = np.round(y_pred_prob).astype(int)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy:', accuracy)
print('F1-score:', f1)

251/251 [==============================] - 3s 8ms/step
Accuracy: 0.7318108074379134
F1-score: 0.69470095183975


In [ ]:

# Plot the learning curves
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

NameError: ignored